In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import matplotlib.cm as cm

from math import sin,cos,acos,sqrt,pi, atan2

import pandas as pd

%matplotlib

Using matplotlib backend: TkAgg


In [2]:
pd.__version__

'1.1.4'

In [3]:
# barycentric coords for triangle (-0.5,0),(0.5,0),(0,sqrt(3)/2)
def barycentricCoords(p):
    '''
        input: 'p'is are the position vector of the form [x,y]'. 
        output: l1,l2,l3 are the barycentric co-ordsinates.
    
        ex:
        barycentricCoords([1,2])
        (-1.6547005383792517, 0.3452994616207483, 2.3094010767585034)
        
        D'une maniere generale
        p1=(x1,y1), p2=(x2,y2) p3=(x3,y3)
        T= [[x1-x3,x2-x3],[y1-y3,y2-y3]]
        (l1,l2) = T^(-1) . ( (x,y)-p3 )
        l3 = 1-l2-l3
    '''
    x,y = p[0],p[1]
    # l3*sqrt(3)/2 = y
    l3 = y*2./sqrt(3.)
    # l1 + l2 + l3 = 1
    # 0.5*(l2 - l1) = x
    l2 = x + 0.5*(1 - l3)
    l1 = 1 - l2 - l3
    return l1,l2,l3

In [4]:
def scalProd(p1,p2):
    '''
        input: p1 and p2 are the vetors of form [x0,x1,...,xn]'
        output: is the scalar product of p1 and p2.
    '''
    return sum([p1[i]*p2[i] for i in range(len(p1))])

In [5]:
def slerp(p0,p1,t):
    '''
        program outputs the spherical linear interpolation 
        of arc defined by p0, p1(around origin).  
        
        input: t=0 -> p0, t=1 -> p1. 
                p0 and p1 are the vetors of form [x,y,z]
        
        output: interpolated coordinates.
        
        https://en.wikipedia.org/wiki/Slerp
        
    '''
    assert abs(scalProd(p0,p0) - scalProd(p1,p1)) < 1e-7
    ang0Cos = scalProd(p0,p1)/scalProd(p0,p0)
    ang0Sin = sqrt(1 - ang0Cos*ang0Cos)
    ang0 = atan2(ang0Sin,ang0Cos)
    l0 = sin((1-t)*ang0)
    l1 = sin(t    *ang0)
    return np.array([(l0*p0[i] + l1*p1[i])/ang0Sin for i in range(len(p0))])


In [6]:
# map 2D point p to spherical triangle s1,s2,s3 (3D vectors of equal length)
def mapGridpoint2Sogere(p,s1,s2,s3):
    '''
        program outputs the coordinate array of the projection of the input
        coordinates on the unit sphere.   
        inputs:
            - 'p' is the coordinate array of the planer verticies of the closed 
                shape to be projected in the form [x,y,z]'.
            - 's1','s2' and 's3' are the vectors defining plane of the co-ordinates 
                to be projected. 
        output: is the coordinate array of the projected face on the unit sphere.
        
        ex. mapGidpoint2Sogere([0,0.5,0.5],[1,0,0]',[0,1,0]',[0,0,1]')
    '''
    l1,l2,l3 = barycentricCoords(p)
    if abs(l3-1) < 1e-10: return s3
    l2s = l2/(l1+l2)
    p12 = slerp(s1,s2,l2s)
    return slerp(p12,s3,l3)

In [7]:
mapGridpoint2Sogere([0,0.5,0.5],[1,0,0],[0,1,0],[0,0,1])

array([0.43571249, 0.43571249, 0.78759714])

In [8]:
def hexagon(x,y,th,scale,opt, fact=0):
    '''
        this program creates the hexagon of given configuration and size.
        inputs: 
            - x and y are the rectangular coordinates of the center of the hexagon
            - th is the rotation angle measured anticlockwise positive 
                from positive x axis
            - scale : is a contraction/dilation factor
            - opt: 1 full hexagon. 2 half hexagon

            output: planar hexagon (complete/truncated) orientated 
        example:
            hexagon(0,0,np.pi/6,0.5,1)
    '''
    # rotation matrx with scale (th>0 the transformation is anti-clockwise)
    rot_mat = scale * np.array([[np.cos(th), -np.sin(th)],
                    [np.sin(th), np.cos(th)]])
    if opt == 1:
        '''
            Hexagone complet
                                   Y
                  0                ^
            1           5          I
                                   I--- > X
            2           4
                  3     
        ''' 
        hex = np.zeros((2,6))
        hex[0,:]= np.array([np.sin(i*np.pi/3) for i in range(6)]) # X-coord
        hex[1,:]= np.array([np.cos(i*np.pi/3) for i in range(6)]) # Y-coord
    
    elif opt == 2:
        '''
            Hexagone tronque
            
                    2               ^
             3             1         I
                                     I
             4             0         I--- >
        ''' 
        hex = np.zeros((2,5))
        hex[0,:]= np.array([sqrt(3)/2,sqrt(3)/2,0,-sqrt(3)/2,-sqrt(3)/2]) # X-ccod
        hex[1,:]= np.array([0,0.5,1,0.5,0]) # Y-coord

    elif opt == 3:
        # point 0 et 1 sont modifiers par rapport au type 2
        hex = np.zeros((2,5))
        hex[0,:]= np.array([sqrt(3)/2-fact,sqrt(3)/2-fact,0,-sqrt(3)/2,-sqrt(3)/2]) # X-ccod
        hex[1,:]= np.array([0,1/2+fact/sqrt(3),1,0.5,0]) # Y-coord

    elif opt == 4:
        # point 3 et 4 sont modifiers par rapport au type 2
        hex = np.zeros((2,5))
        hex[0,:]= np.array([sqrt(3)/2,sqrt(3)/2,0,-sqrt(3)/2+fact,-sqrt(3)/2+fact]) # X-ccod
        hex[1,:]= np.array([0,0.5,1,1/2+fact/sqrt(3),0]) # Y-coord


    hex = np.matmul(rot_mat,hex)
 
    hex[0,:]= x+hex[0,:] 
    hex[1,:]= y+hex[1,:]
    
    return hex

In [9]:
hexagon(0,0,np.pi/6,0.5,1)

array([[-2.50000000e-01,  2.50000000e-01,  5.00000000e-01,
         2.50000000e-01, -2.50000000e-01, -5.00000000e-01],
       [ 4.33012702e-01,  4.33012702e-01,  9.61481343e-17,
        -4.33012702e-01, -4.33012702e-01,  5.55111512e-17]])

In [10]:
def getIcosaedreVertices():
    """
        outputs location of the icosaedre vertices 3D points
    """
    #golden ratio
    phi = 0.5*(1+sqrt(5)) 
    
    topPoints = \
        [(phi,1,0)]+\
        [(phi,-1,0)]+\
        [(1,0,-phi)]+\
        [(0,phi,-1)]+\
        [(0,phi,1)]+\
        [(1,0,phi)]
    
    topPoints = np.array(topPoints)
    # rot clockwise arround Z pour amener le point 1 en position (1,0,0)
    sinth = 1/sqrt(1+phi**2)
    costh = phi*sinth
    scale = 1/sqrt(1+phi**2)
    rot_mat = scale*np.array([[costh,sinth,0],
                        [-sinth, costh,0],
                        [0,0,1]])
    
    for i in range(len(topPoints)):
        topPoints[i,:] = np.matmul(rot_mat,topPoints[i,:])

    # change de repere
    # X' = -Y, Y'=-Z, Z'=X
    tmp = np.zeros_like(topPoints)
    for i in range(topPoints.shape[0]):
        tmp[i,0] = -topPoints[i,1]
        tmp[i,1] = -topPoints[i,2]
        tmp[i,2] =  topPoints[i,0]
    topPoints = tmp
    
    # points du bas de l'icosaedre
    bottomPoints = np.zeros_like(topPoints)
    for i in range(bottomPoints.shape[0]):
        bottomPoints[i,0] = -topPoints[i,0]
        bottomPoints[i,1] =  topPoints[i,1]
        bottomPoints[i,2] = -topPoints[i,2]

    # icosaedre vertices
    icoPoints=np.vstack((topPoints,bottomPoints))
    
    #return
    return icoPoints

In [65]:
icoPoints = getIcosaedreVertices()

In [11]:
def getProjectedFace(hexag,icoPoints,u,v,w):
    """
        outputs the coordinates of projected face on the plane 
        defined by tips of the vectors u,v and w on the unit radius sphere.  
 
         Inputs:

            _ 'hexag' is the coordinate array of the planer verticies of the closed 
             shape to be projected in the form [x,y,z]'.
            _ 'u','v' and 'w' are the vectors defining plane to be projected on 
                 the sphere.
            _ 'icoPoints': icosedre 3D vertices
    
    """
    
    n = hexag.shape[1]
    face = np.zeros((3,n))
    # projecting the input hexagonal mesh on the sphere
    for i in range(n):
        face[:,i] = mapGridpoint2Sogere(hexag[:,i],
                                        icoPoints[u,:],
                                        icoPoints[v,:],
                                        icoPoints[w,:])
    
    return face


In [12]:
def getProjectedPt(p,icoPoints,u,v,w):
    """
    p: 2D point location
    """
    return mapGridpoint2Sogere(p,
                                icoPoints[u,:],
                                icoPoints[v,:],
                                icoPoints[w,:])

In [13]:
def plot_icoVertices():
    s,c = 2/sqrt(5),1/sqrt(5)
    topPoints = [(0,0,1)] + [(s*cos(i*2*pi/5.), s*sin(i*2*pi/5.), c) for i in range(5)]
    bottomPoints = [(-x,y,-z) for (x,y,z) in topPoints]
    icoPoints = topPoints + bottomPoints
    icoPoints = np.array(icoPoints)
    
    print('plot_icoVertices:\n',icoPoints)
    
    fig = plt.figure()
    ax = Axes3D(fig)
    topPoints=np.array(topPoints)
    bottomPoints=np.array(bottomPoints)
    ax.scatter(topPoints[:,0],topPoints[:,1],topPoints[:,2], c='blue')
    ax.scatter(bottomPoints[:,0],bottomPoints[:,1],bottomPoints[:,2], c='red')
    ax.set_xlabel(r'$X$', fontsize=20)
    ax.set_ylabel(r'$Y$', fontsize=20)
    ax.set_zlabel(r'$Z$', fontsize=20)
    plt.show()

<img src="./icosaedre_vertices.png">

In [14]:
def plot_ico3():
    phi = 0.5*(1+sqrt(5)) 

    topPoints = \
        [(phi,1,0)]+\
        [(phi,-1,0)]+\
        [(1,0,-phi)]+\
        [(0,phi,-1)]+\
        [(0,phi,1)]+\
        [(1,0,phi)]
    
    topPoints = np.array(topPoints)
    # rot clockwise arround Z to put point 1 in (1,0,0) position in the X,Y,Z frame
    sinth = 1/sqrt(1+phi**2)
    costh = phi*sinth
    scale = 1/sqrt(1+phi**2)
    rot_mat = scale*np.array([[costh,sinth,0],
                        [-sinth, costh,0],
                        [0,0,1]])
    
    for i in range(len(topPoints)):
        topPoints[i,:] = np.matmul(rot_mat,topPoints[i,:])

    # switch to frame (X',Y',Z')
    # X' = -Y, Y'=-Z, Z'=X
    tmp = np.zeros_like(topPoints)
    for i in range(topPoints.shape[0]):
        tmp[i,0] = -topPoints[i,1]
        tmp[i,1] = -topPoints[i,2]
        tmp[i,2] =  topPoints[i,0]
    topPoints = tmp
    
    # points du bas de l'icosaedre
    bottomPoints = np.zeros_like(topPoints)
    for i in range(bottomPoints.shape[0]):
        bottomPoints[i,0] = -topPoints[i,0]
        bottomPoints[i,1] =  topPoints[i,1]
        bottomPoints[i,2] = -topPoints[i,2]

    icoPoints=np.vstack((topPoints,bottomPoints))
            
    print('ico3:\n',icoPoints)

    fig = plt.figure()
    ax = Axes3D(fig)
    ax.scatter(topPoints[:,0],topPoints[:,1],topPoints[:,2],c='orange')
    ax.scatter(bottomPoints[:,0],bottomPoints[:,1],bottomPoints[:,2],c='cyan')
    ax.set_xlabel(r'$X$', fontsize=20)
    ax.set_ylabel(r'$Y$', fontsize=20)
    ax.set_zlabel(r'$Z$', fontsize=20)
    plt.show()


In [15]:
def Draw_Goldberg_Polyhedron(n,fact=0,return_df=True):
        
    nfaces=20
    
    #DF structure
    if return_df:
        df = pd.DataFrame(columns=['idx','type','center','vertices'])
    
    
    fig = plt.figure()
    ax = Axes3D(fig)
    ax.set_xlabel(r'$X$', fontsize=20)
    ax.set_ylabel(r'$Y$', fontsize=20)
    ax.set_zlabel(r'$Z$', fontsize=20)    
    colors = cm.rainbow(np.linspace(0, 1, nfaces))
    
    #icosedre vertices
    icoPoints = getIcosaedreVertices()
    
    
    x,y = -1/2,0
    # ensemble de tous les triplets de points qui constituent les 20 triangles equilateraux
    icoTriangs = [(0,i+1,(i+1)%5+1) for i in range(5)] +\
             [(6,i+7,(i+1)%5+7) for i in range(5)] +\
             [(i+1,(i+1)%5+1,(7-i)%5+7) for i in range(5)] +\
             [(i+1,(7-i)%5+7,(8-i)%5+7) for i in range(5)]
    icoTriangs=np.array(icoTriangs)

    scale = 1/(n*sqrt(3))
    
    #loop over the 20 faces of the icosaedre
#    for k in [0,1,2,3,4]:
    for k in range(nfaces):
        for i in range(n+1):
            for j in range(n-i+1):
                if i==0:
                    th = -2*pi/3
                    if j==1:
                        opt = 3
                    elif j==n-i-1:
                        opt = 4
                    else:
                        opt = 2
                elif j==n-i: 
                    th = 2*pi/3
                    if i==1:
                        opt = 3
                    elif i==n-1:
                        opt = 4
                    else:
                        opt = 2
                elif (j==0 and i != 0):
                    th = 0
                    if i==1:
                        opt = 4
                    elif i==n-1:
                        opt = 3
                    else:
                        opt = 2
                else:
                    opt = 1
                    th = 0
                #exclude the hexagons at the vertices of the isocele triangle
                if (i!=0 or j!=0) and (i!=0 or j!=n) and (i!=n or j!=0):
                    hexagcenter = np.array([x+i*1/n+j*1/(2*n), y+j*sqrt(3)/(2*n)])
                    hexag = hexagon(hexagcenter[0],hexagcenter[1],th,scale,opt,fact=fact)
                    a = icoTriangs[k,0]
                    b = icoTriangs[k,1]
                    c = icoTriangs[k,2]
                    face = getProjectedFace(hexag,icoPoints,a,b,c)
                    center = getProjectedPt(hexagcenter,icoPoints,a,b,c)
                    xf,yf,zf = face[0,:],face[1,:],face[2,:]
                    vertsf=list(zip(xf,yf,zf))
                    # Dataframe
                    if return_df:
                        df=df.append({'idx':(k,i,j),
                                      'type':opt,
                                      'center':(center[0],center[1],center[2]),
                                      'vertices':vertsf},ignore_index=True)
                    
                    #plot
                    #ax.scatter(xf,yf,zf,s=1)
                    ax.scatter(center[0],center[1],center[2],marker='x',s=10,color=colors[k])
#                    ax.text(center[0]*1.01,center[1]*1.01,center[2]*1.01,'%s/%s' % (str(k),str(opt)), 
#                            size=10, zorder=1, color=colors[k],zdir=(0,0.5,0.5*np.sign(np.mean(zf)-0.5)))
#                    ax.text(center[0]*1.01,center[1]*1.01,center[2]*1.01,'%s/%s/%s' % (str(k),str(j),str(i)), 
#                            size=10, zorder=1, color=colors[k],zdir=(0,0.5,0.5*np.sign(np.mean(zf)-0.5)))

                    ax.add_collection3d(Poly3DCollection([vertsf], facecolors = colors[k], edgecolors='k', linewidths=1, alpha=0.9))

    #eofor
    ax.set_xlim([-1,1])
    ax.set_ylim([-1,1])
    ax.set_zlim([-1,1])
    plt.show()
    # return dataframe
    if return_df:
        return df

In [85]:
icoTriangs = [(0,i+1,(i+1)%5+1) for i in range(5)] +\
             [(6,i+7,(i+1)%5+7) for i in range(5)] +\
             [(i+1,(i+1)%5+1,(7-i)%5+7) for i in range(5)] +\
             [(i+1,(7-i)%5+7,(8-i)%5+7) for i in range(5)]
icoTriangs=np.array(icoTriangs)
icoTriangs

array([[ 0,  1,  2],
       [ 0,  2,  3],
       [ 0,  3,  4],
       [ 0,  4,  5],
       [ 0,  5,  1],
       [ 6,  7,  8],
       [ 6,  8,  9],
       [ 6,  9, 10],
       [ 6, 10, 11],
       [ 6, 11,  7],
       [ 1,  2,  9],
       [ 2,  3,  8],
       [ 3,  4,  7],
       [ 4,  5, 11],
       [ 5,  1, 10],
       [ 1,  9, 10],
       [ 2,  8,  9],
       [ 3,  7,  8],
       [ 4, 11,  7],
       [ 5, 10, 11]])

In [18]:
n=4
df0= Draw_Goldberg_Polyhedron(n,fact=0.0,return_df=True)

In [17]:
len(df0)

240

In [1737]:
#fact=0.1 doit etre Ok, 0.3 c'est pour debugger
Draw_Goldberg_Polyhedron(n,fact=0.3)

,idx,type,center,vertices
0,"(0, 0, 1)",3,"(0.08444400142786841, 0.2598919130077544, 0.96...","[(0.0572479113349012, 0.1761909542530942, 0.98..."
1,"(0, 0, 2)",2,"(0.16245984811645311, 0.5000000000000001, 0.85...","[(0.12464365749479696, 0.3836137326850477, 0.9..."
2,"(0, 0, 3)",4,"(0.22810871757806267, 0.7020464447761632, 0.67...","[(0.1971694431678859, 0.6068251492718142, 0.76..."
3,"(0, 1, 0)",4,"(0.27326652891267167, 0.0, 0.9619383577839173)","[(0.4033553486173643, 0.0, 0.9150434212329841)..."
4,"(0, 1, 1)",1,"(0.37559220582289443, 0.25248481128026146, 0.8...","[(0.3397047863112344, 0.4134568644398938, 0.84..."
...,...,...,...,...
235,"(19, 2, 0)",2,"(0.5877852522924732, -0.8090169943749473, 3.10...","[(0.6408399612512131, -0.7586523001632574, -0...."
236,"(19, 2, 1)",1,"(0.4516777690338741, -0.8535316301556295, -0.2...","[(0.3088198356911398, -0.8846375827832198, -0...."
237,"(19, 2, 2)",2,"(0.26286555605956674, -0.8090169943749473, -0....","[(0.1291518886355933, -0.8439114746223901, -0...."
238,"(19, 3, 0)",3,"(0.6816403771773873, -0.6937804775604491, -0.2...","[(0.7013019753694454, -0.6441236221088784, -0...."


In [19]:
def stat(n):
    typeOfHex=[]
    for i in range(n+1):
        for j in range(n-i+1):
            if i==0:
                opt = 2
                th = -2*pi/3
            elif j==n-i: 
                opt = 2
                th = 2*pi/3
            elif (j==0 and i != 0):
                opt = 2
                th = 0
            else:
                opt = 1
                th = 0

            if (i!=0 or j!=0) and (i!=0 or j!=n) and (i!=n or j!=0):
                typeOfHex.append(opt)
    _,nbre=np.unique(typeOfHex,return_counts=True)
    return nbre

In [20]:
def func2(x):
    a=[i for i in x if x.count(i)==1]
    return a

In [21]:
df0

,idx,type,center,vertices
0,"(0, 0, 1)",3,"(0.08444400142786841, 0.2598919130077544, 0.96...","[(0.04262958722956631, 0.13120037881301283, 0...."
1,"(0, 0, 2)",2,"(0.16245984811645311, 0.5000000000000001, 0.85...","[(0.12464365749479696, 0.3836137326850477, 0.9..."
2,"(0, 0, 3)",4,"(0.22810871757806267, 0.7020464447761632, 0.67...","[(0.1971694431678859, 0.6068251492718142, 0.76..."
3,"(0, 1, 0)",4,"(0.27326652891267167, 0.0, 0.9619383577839173)","[(0.4033553486173643, 0.0, 0.9150434212329841)..."
4,"(0, 1, 1)",1,"(0.37559220582289443, 0.25248481128026146, 0.8...","[(0.3397047863112344, 0.4134568644398938, 0.84..."
...,...,...,...,...
235,"(19, 2, 0)",2,"(0.5877852522924732, -0.8090169943749473, 3.10...","[(0.6408399612512131, -0.7586523001632574, -0...."
236,"(19, 2, 1)",1,"(0.4516777690338741, -0.8535316301556295, -0.2...","[(0.3088198356911398, -0.8846375827832198, -0...."
237,"(19, 2, 2)",2,"(0.26286555605956674, -0.8090169943749473, -0....","[(0.1291518886355933, -0.8439114746223901, -0...."
238,"(19, 3, 0)",3,"(0.6816403771773873, -0.6937804775604491, -0.2...","[(0.7094063048852374, -0.6156420208736807, -0...."


In [22]:
tol=10**(-6)
def myround(x,tol=10**(-6)):
    return tuple(np.round(np.array(x)/tol).astype(int))

In [23]:
df0['center']=df0['center'].map(myround)
df0['vertices']=df0['vertices'].map(myround)

In [24]:
df0

,idx,type,center,vertices
0,"(0, 0, 1)",3,"(84444, 259892, 961938)","([42630, 131200, 990439], [123386, 86635, 9885..."
1,"(0, 0, 2)",2,"(162460, 500000, 850651)","([124644, 383614, 915043], [212854, 338459, 91..."
2,"(0, 0, 3)",4,"(228109, 702046, 674609)","([197169, 606825, 769992], [290514, 563894, 77..."
3,"(0, 1, 0)",4,"(273267, 0, 961938)","([403355, 0, 915043], [397644, 84884, 913605],..."
4,"(0, 1, 1)",1,"(375592, 252485, 891730)","([339705, 413457, 844780], [495798, 332614, 80..."
...,...,...,...,...
235,"(19, 2, 0)",2,"(587785, -809017, 0)","([640840, -758652, -117349], [581684, -796408,..."
236,"(19, 2, 1)",1,"(451678, -853532, -259752)","([308820, -884638, -349352], [419091, -797594,..."
237,"(19, 2, 2)",2,"(262866, -809017, -525731)","([129152, -843911, -520705], [155991, -888299,..."
238,"(19, 3, 0)",3,"(681640, -693780, -232454)","([709406, -615642, -343115], [649201, -647953,..."


In [25]:
g0=df0.groupby('center', as_index=False).agg({'idx':'sum','type':lambda x: list(x),'vertices':'sum'})

In [26]:
g0

,center,idx,type,vertices
0,"(-998989, 0, 44948)","(12, 1, 2)",[1],"([-992251, 0, -124252], [-985772, -159297, -53..."
1,"(-959253, -160622, -232454)","(12, 1, 3, 18, 0, 3)","[3, 4]","([-935787, -81086, -343115], [-957060, 0, -289..."
2,"(-959253, 160622, -232454)","(12, 0, 3, 17, 3, 0)","[4, 3]","([-964375, 237086, -117349], [-985772, 159297,..."
3,"(-951057, -309017, 0)","(12, 2, 2, 18, 0, 2)","[2, 2]","([-964375, -237086, -117349], [-985772, -15929..."
4,"(-951057, 309017, 0)","(12, 0, 2, 17, 2, 0)","[2, 2]","([-919552, 375039, 117349], [-936381, 300816, ..."
...,...,...,...,...
145,"(951333, 165815, -259752)","(15, 2, 1)",[1],"([936771, 20337, -349352], [888063, 152109, -4..."
146,"(958680, -124060, -256012)","(15, 1, 2)",[1],"([905531, -266896, -329818], [893027, -126143,..."
147,"(959253, -160622, 232454)","(14, 3, 1, 15, 0, 1)","[4, 3]","([964375, -237086, 117349], [934363, -307026, ..."
148,"(959253, 160622, 232454)","(10, 0, 1, 15, 1, 0)","[3, 4]","([935787, 81086, 343115], [904000, 144119, 402..."


In [27]:
g0['vertices']=g0['vertices'].map(lambda x: [list(y) for y in x])

In [28]:
g0['vertices']=g0['vertices'].map(lambda x: tuple([tuple(y) for y in x]))

In [29]:
g0['vertices']=g0['vertices'].map(func2)

In [30]:
g0

,center,idx,type,vertices
0,"(-998989, 0, 44948)","(12, 1, 2)",[1],"[(-992251, 0, -124252), (-985772, -159297, -53..."
1,"(-959253, -160622, -232454)","(12, 1, 3, 18, 0, 3)","[3, 4]","[(-957060, 0, -289889), (-992251, 0, -124252),..."
2,"(-959253, 160622, -232454)","(12, 0, 3, 17, 3, 0)","[4, 3]","[(-985772, 159297, -53657), (-992251, 0, -1242..."
3,"(-951057, -309017, 0)","(12, 2, 2, 18, 0, 2)","[2, 2]","[(-985772, -159297, -53657), (-980577, -155157..."
4,"(-951057, 309017, 0)","(12, 0, 2, 17, 2, 0)","[2, 2]","[(-936381, 300816, 180833), (-980577, 155157, ..."
...,...,...,...,...
145,"(951333, 165815, -259752)","(15, 2, 1)",[1],"[(936771, 20337, -349352), (888063, 152109, -4..."
146,"(958680, -124060, -256012)","(15, 1, 2)",[1],"[(905531, -266896, -329818), (893027, -126143,..."
147,"(959253, -160622, 232454)","(14, 3, 1, 15, 0, 1)","[4, 3]","[(934363, -307026, 180833), (890595, -290389, ..."
148,"(959253, 160622, 232454)","(10, 0, 1, 15, 1, 0)","[3, 4]","[(904000, 144119, 402509), (890595, 290389, 35..."


In [31]:
#En principe il doit n'y avoir que des hexagones => tmp=6
g0['Nvertices'] = g0['vertices'].map(lambda x:len(x))

In [32]:
np.unique(g0['Nvertices'])

array([6])

In [33]:
#Des hexagones sont la fusion de 2 polygones appartenant à 2 faces
g0['nfaces'] = g0['idx'].map(lambda x:len(x)//3)

In [34]:
np.unique(g0['nfaces'])

array([1, 2])

In [35]:
g0['nfaces2'] = g0['type'].map(lambda x:len(x))

In [36]:
np.unique(g0['nfaces']==g0['nfaces2'])

array([ True])

In [37]:
np.unique(g0['nfaces'],return_counts=True)

(array([1, 2]), array([60, 90]))

In [38]:
g0=g0.drop(columns=['nfaces2'])

In [39]:
g0

,center,idx,type,vertices,Nvertices,nfaces
0,"(-998989, 0, 44948)","(12, 1, 2)",[1],"[(-992251, 0, -124252), (-985772, -159297, -53...",6,1
1,"(-959253, -160622, -232454)","(12, 1, 3, 18, 0, 3)","[3, 4]","[(-957060, 0, -289889), (-992251, 0, -124252),...",6,2
2,"(-959253, 160622, -232454)","(12, 0, 3, 17, 3, 0)","[4, 3]","[(-985772, 159297, -53657), (-992251, 0, -1242...",6,2
3,"(-951057, -309017, 0)","(12, 2, 2, 18, 0, 2)","[2, 2]","[(-985772, -159297, -53657), (-980577, -155157...",6,2
4,"(-951057, 309017, 0)","(12, 0, 2, 17, 2, 0)","[2, 2]","[(-936381, 300816, 180833), (-980577, 155157, ...",6,2
...,...,...,...,...,...,...
145,"(951333, 165815, -259752)","(15, 2, 1)",[1],"[(936771, 20337, -349352), (888063, 152109, -4...",6,1
146,"(958680, -124060, -256012)","(15, 1, 2)",[1],"[(905531, -266896, -329818), (893027, -126143,...",6,1
147,"(959253, -160622, 232454)","(14, 3, 1, 15, 0, 1)","[4, 3]","[(934363, -307026, 180833), (890595, -290389, ...",6,2
148,"(959253, 160622, 232454)","(10, 0, 1, 15, 1, 0)","[3, 4]","[(904000, 144119, 402509), (890595, 290389, 35...",6,2


Il faut repérer les hexagones resulats de fusions de moities et pour lesquels la liste des points n'est pas connexe 

In [40]:
def cmpdist(x):
    pt2,pt3,pt5=np.array(x[2]),np.array(x[3]),np.array(x[5])
    return ((pt2-pt3)**2).sum()<((pt2-pt5)**2).sum()

In [41]:
g0['good']=g0['vertices'].map(cmpdist)

In [42]:
np.unique(g0['good'],return_counts=True)

(array([False,  True]), array([ 30, 120]))

In [43]:
g0

,center,idx,type,vertices,Nvertices,nfaces,good
0,"(-998989, 0, 44948)","(12, 1, 2)",[1],"[(-992251, 0, -124252), (-985772, -159297, -53...",6,1,True
1,"(-959253, -160622, -232454)","(12, 1, 3, 18, 0, 3)","[3, 4]","[(-957060, 0, -289889), (-992251, 0, -124252),...",6,2,True
2,"(-959253, 160622, -232454)","(12, 0, 3, 17, 3, 0)","[4, 3]","[(-985772, 159297, -53657), (-992251, 0, -1242...",6,2,True
3,"(-951057, -309017, 0)","(12, 2, 2, 18, 0, 2)","[2, 2]","[(-985772, -159297, -53657), (-980577, -155157...",6,2,True
4,"(-951057, 309017, 0)","(12, 0, 2, 17, 2, 0)","[2, 2]","[(-936381, 300816, 180833), (-980577, 155157, ...",6,2,True
...,...,...,...,...,...,...,...
145,"(951333, 165815, -259752)","(15, 2, 1)",[1],"[(936771, 20337, -349352), (888063, 152109, -4...",6,1,True
146,"(958680, -124060, -256012)","(15, 1, 2)",[1],"[(905531, -266896, -329818), (893027, -126143,...",6,1,True
147,"(959253, -160622, 232454)","(14, 3, 1, 15, 0, 1)","[4, 3]","[(934363, -307026, 180833), (890595, -290389, ...",6,2,True
148,"(959253, 160622, 232454)","(10, 0, 1, 15, 1, 0)","[3, 4]","[(904000, 144119, 402509), (890595, 290389, 35...",6,2,True


In [44]:
mask = (g0['good'] == False)
g0_tbm = g0[mask]

In [45]:
def swapt(x):
    a=x
    a[3],a[5]=a[5],a[3]
    return a

In [46]:
g0.loc[mask, 'vertices']= g0_tbm['vertices'].map(swapt)    

In [47]:
g0

,center,idx,type,vertices,Nvertices,nfaces,good
0,"(-998989, 0, 44948)","(12, 1, 2)",[1],"[(-992251, 0, -124252), (-985772, -159297, -53...",6,1,True
1,"(-959253, -160622, -232454)","(12, 1, 3, 18, 0, 3)","[3, 4]","[(-957060, 0, -289889), (-992251, 0, -124252),...",6,2,True
2,"(-959253, 160622, -232454)","(12, 0, 3, 17, 3, 0)","[4, 3]","[(-985772, 159297, -53657), (-992251, 0, -1242...",6,2,True
3,"(-951057, -309017, 0)","(12, 2, 2, 18, 0, 2)","[2, 2]","[(-985772, -159297, -53657), (-980577, -155157...",6,2,True
4,"(-951057, 309017, 0)","(12, 0, 2, 17, 2, 0)","[2, 2]","[(-936381, 300816, 180833), (-980577, 155157, ...",6,2,True
...,...,...,...,...,...,...,...
145,"(951333, 165815, -259752)","(15, 2, 1)",[1],"[(936771, 20337, -349352), (888063, 152109, -4...",6,1,True
146,"(958680, -124060, -256012)","(15, 1, 2)",[1],"[(905531, -266896, -329818), (893027, -126143,...",6,1,True
147,"(959253, -160622, 232454)","(14, 3, 1, 15, 0, 1)","[4, 3]","[(934363, -307026, 180833), (890595, -290389, ...",6,2,True
148,"(959253, 160622, 232454)","(10, 0, 1, 15, 1, 0)","[3, 4]","[(904000, 144119, 402509), (890595, 290389, 35...",6,2,True


In [48]:
g0['good']=g0['vertices'].map(cmpdist)

In [49]:
np.unique(g0['good'],return_counts=True)

(array([ True]), array([150]))

In [50]:
np.unique(g0['type'])

array([list([1]), list([2, 2]), list([3, 3]), list([3, 4]), list([4, 3]),
       list([4, 4])], dtype=object)

In [51]:
def myfloat(x,tol=10**(-6)):
    return tuple(np.array(x)*tol)

In [78]:
tol=10**(-6)

fig = plt.figure()
ax = Axes3D(fig)
ax.set_xlabel(r'$X$', fontsize=20)
ax.set_ylabel(r'$Y$', fontsize=20)
ax.set_zlabel(r'$Z$', fontsize=20)    

nfaces=20

types = g0.loc[:,'type'].values
center = np.stack(g0.loc[:,'center'].values)*tol
ax.scatter(center[:,0],center[:,1],center[:,2],marker='x',s=10)

faces = np.stack(g0.loc[:,'vertices'].values)*tol
for f in range(faces.shape[0]):
    xf,yf,zf = faces[f,:,0],faces[f,:,1],faces[f,:,2]
    vertsf=list(zip(xf,yf,zf))
    if types[f]==[1]:      # hexagones inside icosaedre face
        col='yellow'
    elif types[f]==[2,2]:  # hexagones edges between 2 icosaedre faces
        col='green'
    else:                  # hexagones edges between 2 icosaedre faces neighboors of pentagons
        col='red'
    ax.add_collection3d(Poly3DCollection([vertsf], facecolors = col, edgecolors='k', linewidths=1, alpha=0.9))
    
plt.show()

In [365]:
def xycent(xoff7,yoff7,i,j,n):
    return xoff7+i/n+j/(2*n), yoff7+j*sqrt(3)/n

In [366]:
xoff7=-1/2
yoff7=0
n=4
scale = 1/(n*sqrt(3))
fact=0
tol=10**(-6)

pentaBuild=pd.DataFrame(columns=['idx','face','xyc','th','type'])


#Penta #0 : top
idx0 = (0,1,2,3,4)
for k in idx0:
    info = {
        'idx':idx0,
        'face':k,
        'xyc':xycent(xoff7,yoff7,1,0,n), #type 4
        'th': 0,
        'type': 4
    }
    pentaBuild=pentaBuild.append(info,ignore_index=True)

    

######
#Pentas of the upper ring
######
#Penta #1 :
#cases[0,0],cases[0,1],cases[0,2],cases[0,3] =  expand(xycent(xoff7,yoff7,0,1,n),(-2*pi/3,3))
#cases[1,0],cases[1,1],cases[1,2],cases[1,3] =  expand(xycent(xoff7,yoff7,1,n-1,n),(2*pi/3,3))
#cases[2,0],cases[2,1],cases[2,2],cases[2,3] =  expand(xycent(xoff7,yoff7,n-1,0,n),(0,3))
#cases[0,0],cases[0,1],cases[0,2],cases[0,3] =  expand(xycent(xoff7,yoff7,0,n-1,n),(-2*pi/3,4))
#cases[0,0],cases[0,1],cases[0,2],cases[0,3] =  expand(xycent(xoff7,yoff7,n-1,1,n),(2*pi/3,4))
#cases[0,0],cases[0,1],cases[0,2],cases[0,3] =  expand(xycent(xoff7,yoff7,1,0,n),(0,4))

idx0 = (0,1,11,16,10)
infos=[]
"""
infos.append({
        'idx':idx0,
        'face':0,
        'xyc':xycent(xoff7,yoff7,0,1,n),
        'th': -2*pi/3,
        'type': 3
    })
infos.append({
        'idx':idx0,
        'face':0,
        'xyc':xycent(xoff7,yoff7,1,n-1,n),
        'th': 2*pi/3,
        'type': 3
    })

infos.append({
        'idx':idx0,
        'face':0,
        'xyc':xycent(xoff7,yoff7,n-1,0,n),
        'th': 0,
        'type': 3
    })

infos.append({
        'idx':idx0,
        'face':0,
        'xyc':xycent(xoff7,yoff7,0,n-1,n),
        'th': -2*pi/3,
        'type': 4
    })
"""
infos.append({
        'idx':idx0,
        'face':0,
        'xyc':xycent(xoff7,yoff7,n-1,1,n),
        'th': 2*pi/3,
        'type': 4
    })



infos.append({
        'idx':idx0,
        'face':1,
        'xyc':xycent(xoff7,yoff7,n-1,0,n),
        'th': 0,
        'type': 3
    })

infos.append({
        'idx':idx0,
        'face':11,
        'xyc':xycent(xoff7,yoff7,1,0,n),
        'th':0, #-2*pi/3
        'type':4
    })

infos.append({
        'idx':idx0,
        'face':16,
        'xyc':xycent(xoff7,yoff7,1,0,n),
        'th':0, #-2*pi/3
        'type':4
    })

infos.append({
        'idx':idx0,
        'face':10,
        'xyc':xycent(xoff7,yoff7,n-1,0,n),
        'th': 0, #0
        'type':3
    })

for info in infos:
    pentaBuild=pentaBuild.append(info,ignore_index=True)

"""


#Penta #2 : 1,2,11,12,17
idx0 = (1,2,12,17,11)
infos=[]
infos.append({
        'idx':idx0,
        'face':1,
        'xyc':xycent(xoff7,yoff7,1,n-1,n),
        'th':2*pi/3
    })
infos.append({
        'idx':idx0,
        'face':2,
        'xyc':xycent(xoff7,yoff7,n-1,0,n),
        'th':0
    })
infos.append({
        'idx':idx0,
        'face':12,
        'xyc':xycent(xoff7,yoff7,0,1,n),
        'th':-2*pi/3
    })
infos.append({
        'idx':idx0,
        'face':17,
        'xyc':xycent(xoff7,yoff7,0,1,n),
        'th':-2*pi/3
    })
infos.append({
        'idx':idx0,
        'face':11,
        'xyc':xycent(xoff7,yoff7,n-1,0,n),
        'th':0
    })

for info in infos:
    pentaBuild = pentaBuild.append(info,ignore_index=True)

    
#Penta #3 :
idx0 = (2,3,13,18,12)
infos=[]
infos.append({
        'idx':idx0,
        'face':2,
        'xyc':xycent(xoff7,yoff7,1,n-1,n),
        'th':2*pi/3
    })
infos.append({
        'idx':idx0,
        'face':3,
        'xyc':xycent(xoff7,yoff7,n-1,0,n),
        'th':0,
    })
infos.append({
        'idx':idx0,
        'face':13,
        'xyc':xycent(xoff7,yoff7,0,1,n),
        'th':-2*pi/3
    })
infos.append({
        'idx':idx0,
        'face':18,
        'xyc':xycent(xoff7,yoff7,0,1,n),
        'th':-2*pi/3
    })
infos.append({
        'idx':idx0,
        'face':12,
        'xyc':xycent(xoff7,yoff7,n-1,0,n),
        'th':0
    })
for info in infos:
    pentaBuild = pentaBuild.append(info,ignore_index=True)

       
#Penta #4 :
idx0 = (3,4,14,19,13)
infos=[]
infos.append({
        'idx':idx0,
        'face':3,
        'xyc':xycent(xoff7,yoff7,1,n-1,n),
        'th':2*pi/3
    })
infos.append({
        'idx':idx0,
        'face':4,
        'xyc':xycent(xoff7,yoff7,n-1,0,n),
        'th':0
    })
infos.append({
        'idx':idx0,
        'face':14,
        'xyc':xycent(xoff7,yoff7,0,1,n),
        'th':-2*pi/3
    })
infos.append({
        'idx':idx0,
        'face':19,
        'xyc':xycent(xoff7,yoff7,0,1,n),
        'th':-2*pi/3
    })
infos.append({
        'idx':idx0,
        'face':13,
        'xyc':xycent(xoff7,yoff7,n-1,0,n),
        'th':0
    })
for info in infos:
    pentaBuild = pentaBuild.append(info,ignore_index=True)


#cases[0,0],cases[0,1],cases[0,2],cases[0,3] =  expand(xycent(xoff7,yoff7,0,1,n),(-2*pi/3,3))
#cases[1,0],cases[1,1],cases[1,2],cases[1,3] =  expand(xycent(xoff7,yoff7,1,n-1,n),(2*pi/3,3))
#cases[2,0],cases[2,1],cases[2,2],cases[2,3] =  expand(xycent(xoff7,yoff7,n-1,0,n),(0,3))
    
#Penta #5 : 
idx0 = (4,0,10,15,14)
infos=[]
infos.append({
        'idx':idx0,
        'face':4,
        'xyc':xycent(xoff7,yoff7,1,n-1,n),
        'th':2*pi/3
    })
infos.append({
        'idx':idx0,
        'face':0,
        'xyc':xycent(xoff7,yoff7,n-1,0,n),
        'th':0
    })
infos.append({
        'idx':idx0,
        'face':10,
        'xyc':xycent(xoff7,yoff7,0,1,n),
        'th':-2*pi/3
    })
infos.append({
        'idx':idx0,
        'face':15,
        'xyc':xycent(xoff7,yoff7,0,1,n),
        'th':-2*pi/3
    })
infos.append({
        'idx':idx0,
        'face':14,
        'xyc':xycent(xoff7,yoff7,n-1,0,n),
        'th':0
    })
for info in infos:
    pentaBuild = pentaBuild.append(info,ignore_index=True)

######
#Pentas of the lower ring
######

#cases[0,0],cases[0,1],cases[0,2],cases[0,3] =  expand(xycent(xoff7,yoff7,0,1,n),(-2*pi/3,3))
#cases[1,0],cases[1,1],cases[1,2],cases[1,3] =  expand(xycent(xoff7,yoff7,1,n-1,n),(2*pi/3,3))
#cases[2,0],cases[2,1],cases[2,2],cases[2,3] =  expand(xycent(xoff7,yoff7,n-1,0,n),(0,3))
    

#Penta #6 :
idx0 = (6,7,15,10,16)
infos=[]
infos.append({
        'idx':idx0,
        'face':6,
        'xyc':xycent(xoff7,yoff7,1,n-1,n),
        'th':2*pi/3
    })
infos.append({
        'idx':idx0,
        'face':7,
        'xyc':xycent(xoff7,yoff7,n-1,0,n),
        'th':0
    })
infos.append({
        'idx':idx0,
        'face':15,
        'xyc':xycent(xoff7,yoff7,n-1,0,n),
        'th':0
    })
infos.append({
        'idx':idx0,
        'face':10,
        'xyc':xycent(xoff7,yoff7,1,n-1,n),
        'th':2*pi/3
    })
infos.append({
        'idx':idx0,
        'face':16,
        'xyc':xycent(xoff7,yoff7,1,n-1,n),
        'th':2*pi/3
    })
for info in infos:
    pentaBuild = pentaBuild.append(info,ignore_index=True)

#cases[0,0],cases[0,1],cases[0,2],cases[0,3] =  expand(xycent(xoff7,yoff7,0,1,n),(-2*pi/3,3))
#cases[1,0],cases[1,1],cases[1,2],cases[1,3] =  expand(xycent(xoff7,yoff7,1,n-1,n),(2*pi/3,3))
#cases[2,0],cases[2,1],cases[2,2],cases[2,3] =  expand(xycent(xoff7,yoff7,n-1,0,n),(0,3))
    

#Penta #7 :
idx0 = (5,6,16,11,17)
infos=[]
infos.append({
        'idx':idx0,
        'face':5,
        'xyc':xycent(xoff7,yoff7,1,n-1,n),
        'th':2*pi/3
    })
infos.append({
        'idx':idx0,
        'face':6,
        'xyc':xycent(xoff7,yoff7,n-1,0,n),
        'th':0
    })
infos.append({
        'idx':idx0,
        'face':16,
        'xyc':xycent(xoff7,yoff7,n-1,0,n),
        'th':0
    })
infos.append({
        'idx':idx0,
        'face':11,
        'xyc':xycent(xoff7,yoff7,1,n-1,n),
        'th':2*pi/3
    })
infos.append({
        'idx':idx0,
        'face':17,
        'xyc':xycent(xoff7,yoff7,1,n-1,n),
        'th':2*pi/3
    })
for info in infos:
    pentaBuild = pentaBuild.append(info,ignore_index=True)


#Penta #8 
idx0 = (9,5,17,12,18)
infos=[]
infos.append({
        'idx':idx0,
        'face':9,
        'xyc':xycent(xoff7,yoff7,1,n-1,n),
        'th':2*pi/3
    })
infos.append({
        'idx':idx0,
        'face':5,
        'xyc':xycent(xoff7,yoff7,n-1,0,n),
        'th':0
    })
infos.append({
        'idx':idx0,
        'face':17,
        'xyc':xycent(xoff7,yoff7,n-1,0,n),
        'th':0
    })
infos.append({
        'idx':idx0,
        'face':12,
        'xyc':xycent(xoff7,yoff7,1,n-1,n),
        'th':2*pi/3
    })
infos.append({
        'idx':idx0,
        'face':18,
        'xyc':xycent(xoff7,yoff7,1,n-1,n),
        'th':2*pi/3
    })
for info in infos:
    pentaBuild = pentaBuild.append(info,ignore_index=True)


#Penta #9 : 8,9,19,18,13
idx0 = (8,9,18,13,19)
infos=[]
infos.append({
        'idx':idx0,
        'face':8,
        'xyc':xycent(xoff7,yoff7,1,n-1,n),
        'th':2*pi/3
    })
infos.append({
        'idx':idx0,
        'face':9,
        'xyc':xycent(xoff7,yoff7,n-1,0,n),
        'th':0
    })
infos.append({
        'idx':idx0,
        'face':18,
        'xyc':xycent(xoff7,yoff7,n-1,0,n),
        'th':0
    })
infos.append({
        'idx':idx0,
        'face':13,
        'xyc':xycent(xoff7,yoff7,1,n-1,n),
        'th':2*pi/3
    })
infos.append({
        'idx':idx0,
        'face':19,
        'xyc':xycent(xoff7,yoff7,1,n-1,n),
        'th':2*pi/3
    })
for info in infos:
    pentaBuild = pentaBuild.append(info,ignore_index=True)


#Penta #10
idx0 = (7,8,19,14,15)
infos=[]
infos.append({
        'idx':idx0,
        'face':7,
        'xyc':xycent(xoff7,yoff7,1,n-1,n),
        'th':2*pi/3
    })
infos.append({
        'idx':idx0,
        'face':8,
        'xyc':xycent(xoff7,yoff7,n-1,0,n),
        'th':0
    })
infos.append({
        'idx':idx0,
        'face':19,
        'xyc':xycent(xoff7,yoff7,n-1,0,n),
        'th':0
    })
infos.append({
        'idx':idx0,
        'face':14,
        'xyc':xycent(xoff7,yoff7,1,n-1,n),
        'th':2*pi/3
    })
infos.append({
        'idx':idx0,
        'face':15,
        'xyc':xycent(xoff7,yoff7,1,n-1,n),
        'th':2*pi/3
    })
for info in infos:
    pentaBuild = pentaBuild.append(info,ignore_index=True)
"""

#Penta #11  : bottom
idx0 = (5,6,7,8,9)
for k in idx0:
    info = {
        'idx':idx0,
        'face':k,
        'xyc':xycent(xoff7,yoff7,1,0,n), #type 4
        'th':0, # -2*pi/3
        'type':4
    }
    pentaBuild=pentaBuild.append(info,ignore_index=True)




In [367]:
pentaBuild = pentaBuild.drop('face',axis=1)

In [368]:
fig = plt.figure()
ax = Axes3D(fig)
ax.set_xlabel(r'$X$', fontsize=20)
ax.set_ylabel(r'$Y$', fontsize=20)
ax.set_zlabel(r'$Z$', fontsize=20)    

pentas = np.unique(pentaBuild['idx'].values)

print("pentas:",pentas)
#colors = ['red','blue','green','darkred','deepskyblue','forestgreen',
#         'pink','purple','lime','chocolate','darkviolet','chartreuse']
 
colors = ["black","red","blue","green","deepskyblue"]
    
for i in range(len(pentas)):
    idx0 = pentas[i]
    dftmp=pentaBuild[pentaBuild['idx']==idx0]
    print(dftmp)
    pts3d = []
    for ik,k in enumerate(idx0):
        a = icoTriangs[k,0]
        b = icoTriangs[k,1]
        c = icoTriangs[k,2]
        th = dftmp['th'].values[ik]
        xc,yc=dftmp['xyc'].values[ik]
        case=dftmp['type'].values[ik]
        if case == 3:
            pt2d = np.array([sqrt(3)/2-fact,1/2+fact/sqrt(3)]) # type 3 
        else:
            pt2d = np.array([-sqrt(3)/2+fact,1/2+fact/sqrt(3)]) # type 4 

        rot_mat = scale * np.array([[np.cos(th), -np.sin(th)],
                                    [np.sin(th), np.cos(th)]])

        pt2d = np.matmul(rot_mat,pt2d)
        pt2d[0] += xc
        pt2d[1] += yc

        pt3d = getProjectedPt(pt2d,icoPoints,a,b,c)
        pts3d.append(pt3d)

    pts3d = np.array(list(pts3d))
    ax.scatter(pts3d[:,0],pts3d[:,1],pts3d[:,2],marker='o',s=15,color=colors)
    
    vertsf=list(zip(pts3d[:,0],pts3d[:,1],pts3d[:,2]))
    ax.add_collection3d(Poly3DCollection([vertsf], facecolors = 'purple', edgecolors='k', linewidths=1, alpha=0.2))
 
    
    
ax.set_xlim([-1,1])
ax.set_ylim([-1,1])
ax.set_zlim([-1,1])
plt.show()

pentas: [(0, 1, 2, 3, 4) (0, 1, 11, 16, 10) (5, 6, 7, 8, 9)]
               idx           xyc th type
0  (0, 1, 2, 3, 4)  (-0.25, 0.0)  0    4
1  (0, 1, 2, 3, 4)  (-0.25, 0.0)  0    4
2  (0, 1, 2, 3, 4)  (-0.25, 0.0)  0    4
3  (0, 1, 2, 3, 4)  (-0.25, 0.0)  0    4
4  (0, 1, 2, 3, 4)  (-0.25, 0.0)  0    4
                  idx                          xyc      th type
5  (0, 1, 11, 16, 10)  (0.375, 0.4330127018922193)  2.0944    4
6  (0, 1, 11, 16, 10)                  (0.25, 0.0)       0    3
7  (0, 1, 11, 16, 10)                 (-0.25, 0.0)       0    4
8  (0, 1, 11, 16, 10)                 (-0.25, 0.0)       0    4
9  (0, 1, 11, 16, 10)                  (0.25, 0.0)       0    3
                idx           xyc th type
10  (5, 6, 7, 8, 9)  (-0.25, 0.0)  0    4
11  (5, 6, 7, 8, 9)  (-0.25, 0.0)  0    4
12  (5, 6, 7, 8, 9)  (-0.25, 0.0)  0    4
13  (5, 6, 7, 8, 9)  (-0.25, 0.0)  0    4
14  (5, 6, 7, 8, 9)  (-0.25, 0.0)  0    4


In [212]:
pentaBuild.head(10)

,idx,xyc,th,type
0,"(0, 1, 2, 3, 4)","(-0.25, 0.0)",0,4.0
1,"(0, 1, 2, 3, 4)","(-0.25, 0.0)",0,4.0
2,"(0, 1, 2, 3, 4)","(-0.25, 0.0)",0,4.0
3,"(0, 1, 2, 3, 4)","(-0.25, 0.0)",0,4.0
4,"(0, 1, 2, 3, 4)","(-0.25, 0.0)",0,4.0
5,"(0, 1, 11, 16, 10)","(-0.375, 0.4330127018922193)",-2.0944,3.0
6,"(0, 1, 11, 16, 10)","(-0.125, 1.299038105676658)",-2.0944,4.0
7,"(0, 1, 11, 16, 10)","(-0.25, 0.0)",0,4.0
8,"(0, 1, 11, 16, 10)","(-0.25, 0.0)",0,4.0
9,"(0, 1, 11, 16, 10)","(-0.125, 1.299038105676658)",-2.0944,4.0


Test de groupby idx de pentaBuild pour ensuite calculer le centre de chaque pentagone

In [149]:
pentaBuild=pentaBuild.groupby('idx',as_index=False).agg(lambda x: x.tolist())

In [150]:
pentaBuild

,idx,xyc,th
0,"(0, 1, 2, 3, 4)","[(-0.25, 0.0), (-0.25, 0.0), (-0.25, 0.0), (-0...","[0, 0, 0, 0, 0]"
1,"(5, 6, 7, 8, 9)","[(-0.25, 0.0), (-0.25, 0.0), (-0.25, 0.0), (-0...","[0, 0, 0, 0, 0]"


In [151]:
pentaBuild['info']=[[(*a, b) for a, b in zip(x, y)] for x, y in zip(pentaBuild['xyc'],pentaBuild['th'])]
pentaBuild=pentaBuild.drop(['xyc','th'],axis=1)

In [152]:
pentaBuild

,idx,info
0,"(0, 1, 2, 3, 4)","[(-0.25, 0.0, 0), (-0.25, 0.0, 0), (-0.25, 0.0..."
1,"(5, 6, 7, 8, 9)","[(-0.25, 0.0, 0), (-0.25, 0.0, 0), (-0.25, 0.0..."


In [153]:
for row in pentaBuild.itertuples():
#    print(row.idx,row.info)
    idx0 = row.idx
    info0 = np.array(row.info)
    for ik,k in enumerate(idx0):
        xc,yc,th=info0[ik][0],info0[ik][1],info0[ik][2]
        #print("ik:",ik,xc,yc,th)

In [154]:
pentaBuild

,idx,info
0,"(0, 1, 2, 3, 4)","[(-0.25, 0.0, 0), (-0.25, 0.0, 0), (-0.25, 0.0..."
1,"(5, 6, 7, 8, 9)","[(-0.25, 0.0, 0), (-0.25, 0.0, 0), (-0.25, 0.0..."


In [156]:
fig = plt.figure()
ax = Axes3D(fig)
ax.set_xlabel(r'$X$', fontsize=20)
ax.set_ylabel(r'$Y$', fontsize=20)
ax.set_zlabel(r'$Z$', fontsize=20)    


#print("pentas:",pentas)
colors = ['red','blue','green','darkred','deepskyblue','forestgreen',
         'pink','purple','lime','chocolate','darkviolet','chartreuse']

for row in pentaBuild.itertuples():
    def make_pts3d(row):
        idx0 = row.idx
        info0 = np.array(row.info) 
        pts3d = []
        for ik,k in enumerate(idx0):
            a = icoTriangs[k,0]
            b = icoTriangs[k,1]
            c = icoTriangs[k,2]
            xc,yc,th=info0[ik][0],info0[ik][1],info0[ik][2]
            pt2d = np.array([sqrt(3)/2-fact,1/2+fact/sqrt(3)]) # type 3 
            rot_mat = scale * np.array([[np.cos(th), -np.sin(th)],
                                        [np.sin(th), np.cos(th)]])

            pt2d = np.matmul(rot_mat,pt2d)
            pt2d[0] += xc
            pt2d[1] += yc

            pt3d = myround(getProjectedPt(pt2d,icoPoints,a,b,c))
            pts3d.append(pt3d)

        pts3d = np.array(list(pts3d))*tol
        ax.scatter(pts3d[:,0],pts3d[:,1],pts3d[:,2],marker='o',s=15,color='k')

        vertsf=list(zip(pts3d[:,0],pts3d[:,1],pts3d[:,2]))
        ax.add_collection3ad(Poly3DCollection([vertsf], facecolors = 'purple', edgecolors='k', linewidths=1, alpha=0.2))
        return vertsf
    
    pentaBuild['pts3d'] =pentaBuild.apply(make_pts3d, axis=1)
    
ax.set_xlim([-1,1])
ax.set_ylim([-1,1])
ax.set_zlim([-1,1])
plt.show()

AttributeError: 'Axes3D' object has no attribute 'add_collection3ad'

In [1823]:
dfpenta= pentaBuild[['idx','pts3d']]

In [1824]:
dfpenta

,idx,pts3d
0,"(0, 1, 2, 3, 4)","[(0.123386, 0.08663499999999999, 0.98857), (-0..."
1,"(0, 1, 11, 16, 10)","[(0.347887, 0.749751, 0.562893), (0.173788, 0...."
2,"(1, 2, 12, 17, 11)","[(-0.605552, 0.562546, 0.562893), (-0.721617, ..."
3,"(2, 3, 13, 18, 12)","[(-0.722139, -0.402078, 0.562893), (-0.619772,..."
4,"(3, 4, 14, 19, 13)","[(0.159246, -0.811044, 0.562893), (0.338576999..."
5,"(4, 0, 10, 15, 14)","[(0.820558, -0.099175, 0.562893), (0.829024, 0..."
6,"(5, 6, 7, 8, 9)","[(-0.123386, 0.08663499999999999, -0.98857), (..."
7,"(5, 6, 16, 11, 17)","[(-0.347887, 0.749751, -0.562893), (-0.173788,..."
8,"(6, 7, 15, 10, 16)","[(0.605552, 0.562546, -0.562893), (0.721617, 0..."
9,"(7, 8, 19, 14, 15)","[(0.722139, -0.402078, -0.562893), (0.619772, ..."


In [1827]:
fig = plt.figure()
ax = Axes3D(fig)
ax.set_xlabel(r'$X$', fontsize=20)
ax.set_ylabel(r'$Y$', fontsize=20)
ax.set_zlabel(r'$Z$', fontsize=20)    
#print("pentas:",pentas)
colors = ['red','blue','green','darkred','deepskyblue','forestgreen',
         'pink','purple','lime','chocolate','darkviolet','chartreuse']
for row in dfpenta.itertuples():
    vertsf=row.pts3d
    ax.add_collection3d(Poly3DCollection([vertsf], facecolors = 'purple', edgecolors='k', linewidths=1, alpha=0.2))

ax.set_xlim3d(-1,1)
ax.set_ylim3d(-1,1)
ax.set_zlim3d(-1,1)
    
plt.show()

In [1848]:
dfpenta['xyzc']=dfpenta['pts3d']\
                    .map(lambda x: np.array(x).mean(axis=0))\
                    .map(lambda x: x/sqrt(sum(x*x)))

In [1851]:
dfpenta=dfpenta.rename(columns={'pts3d':'vertices','xyzc':'center'})

In [1852]:
dfpenta

,idx,vertices,center
0,"(0, 1, 2, 3, 4)","[(0.123386, 0.08663499999999999, 0.98857), (-0...","[0.0, 0.0, 1.0]"
1,"(0, 1, 11, 16, 10)","[(0.347887, 0.749751, 0.562893), (0.173788, 0....","[0.2777589387431412, 0.8482349260125831, 0.450..."
2,"(1, 2, 12, 17, 11)","[(-0.605552, 0.562546, 0.562893), (-0.721617, ...","[-0.7208870909376458, 0.5262834731781656, 0.45..."
3,"(2, 3, 13, 18, 12)","[(-0.722139, -0.402078, 0.562893), (-0.619772,...","[-0.7232916691387643, -0.5229738982719108, 0.4..."
4,"(3, 4, 14, 19, 13)","[(0.159246, -0.811044, 0.562893), (0.338576999...","[0.2738684794799379, -0.8494989872752924, 0.45..."
5,"(4, 0, 10, 15, 14)","[(0.820558, -0.099175, 0.562893), (0.829024, 0...","[0.892551581527721, -0.0020453458982151408, 0...."
6,"(5, 6, 7, 8, 9)","[(-0.123386, 0.08663499999999999, -0.98857), (...","[0.0, 0.0, -1.0]"
7,"(5, 6, 16, 11, 17)","[(-0.347887, 0.749751, -0.562893), (-0.173788,...","[-0.282522366258839, 0.8495807858689473, -0.44..."
8,"(6, 7, 15, 10, 16)","[(0.605552, 0.562546, -0.562893), (0.721617, 0...","[0.7206950666700204, 0.5312297333272566, -0.44..."
9,"(7, 8, 19, 14, 15)","[(0.722139, -0.402078, -0.562893), (0.619772, ...","[0.7279363832640424, -0.5212628829219892, -0.4..."


In [1853]:
fig = plt.figure()
ax = Axes3D(fig)
ax.set_xlabel(r'$X$', fontsize=20)
ax.set_ylabel(r'$Y$', fontsize=20)
ax.set_zlabel(r'$Z$', fontsize=20)    
#print("pentas:",pentas)
colors = ['red','blue','green','darkred','deepskyblue','forestgreen',
         'pink','purple','lime','chocolate','darkviolet','chartreuse']
for row in dfpenta.itertuples():
    vertsf=row.vertices
    ax.add_collection3d(Poly3DCollection([vertsf], facecolors = 'purple', edgecolors='k', linewidths=1, alpha=0.2))
    xyzc = row.center
    ax.scatter(xyzc[0],xyzc[1],xyzc[2],marker='o',s=15,color='k')

    
ax.set_xlim3d(-1,1)
ax.set_ylim3d(-1,1)
ax.set_zlim3d(-1,1)
    
plt.show()

In [1858]:
dfhexa=g0.drop(['Nvertices','nfaces','good'],axis=1)

In [1861]:
dfhexa=dfhexa[dfhexa.columns[[1,2,3,0]]]

In [1862]:
dfhexa

,idx,type,vertices,center
0,"(12, 1, 2)",[1],"[(-992251, 0, -124252), (-985772, -159297, -53...","(-998989, 0, 44948)"
1,"(12, 1, 3, 18, 0, 3)","[3, 4]","[(-957060, 0, -289889), (-992251, 0, -124252),...","(-959253, -160622, -232454)"
2,"(12, 0, 3, 17, 3, 0)","[4, 3]","[(-985772, 159297, -53657), (-992251, 0, -1242...","(-959253, 160622, -232454)"
3,"(12, 2, 2, 18, 0, 2)","[2, 2]","[(-985772, -159297, -53657), (-980577, -155157...","(-951057, -309017, 0)"
4,"(12, 0, 2, 17, 2, 0)","[2, 2]","[(-936381, 300816, 180833), (-980577, 155157, ...","(-951057, 309017, 0)"
...,...,...,...,...
145,"(15, 2, 1)",[1],"[(936771, 20337, -349352), (888063, 152109, -4...","(951333, 165815, -259752)"
146,"(15, 1, 2)",[1],"[(905531, -266896, -329818), (893027, -126143,...","(958680, -124060, -256012)"
147,"(14, 3, 1, 15, 0, 1)","[4, 3]","[(934363, -307026, 180833), (890595, -290389, ...","(959253, -160622, 232454)"
148,"(10, 0, 1, 15, 1, 0)","[3, 4]","[(904000, 144119, 402509), (890595, 290389, 35...","(959253, 160622, 232454)"


In [1864]:
dfhexa['type']=dfhexa['type'].map(lambda x: 1 if x==[1] else (2 if x==[2,2] else 3))

In [1869]:
dfhexa['vertices']=dfhexa['vertices'].map(lambda x: np.array(x)*tol)

In [1871]:
dfhexa['center']=dfhexa['center'].map(lambda x: np.array(x)*tol)

In [1872]:
dfhexa

,idx,type,vertices,center
0,"(12, 1, 2)",1,"[[-0.992251, 0.0, -0.12425199999999999], [-0.9...","[-0.9989889999999999, 0.0, 0.044947999999999995]"
1,"(12, 1, 3, 18, 0, 3)",3,"[[-0.9570599999999999, 0.0, -0.289889], [-0.99...","[-0.9592529999999999, -0.160622, -0.232454]"
2,"(12, 0, 3, 17, 3, 0)",3,"[[-0.985772, 0.159297, -0.053656999999999996],...","[-0.9592529999999999, 0.160622, -0.232454]"
3,"(12, 2, 2, 18, 0, 2)",2,"[[-0.985772, -0.159297, -0.053656999999999996]...","[-0.9510569999999999, -0.309017, 0.0]"
4,"(12, 0, 2, 17, 2, 0)",2,"[[-0.9363809999999999, 0.300816, 0.180833], [-...","[-0.9510569999999999, 0.309017, 0.0]"
...,...,...,...,...
145,"(15, 2, 1)",1,"[[0.9367709999999999, 0.020336999999999997, -0...","[0.951333, 0.165815, -0.259752]"
146,"(15, 1, 2)",1,"[[0.905531, -0.26689599999999997, -0.329818], ...","[0.95868, -0.12405999999999999, -0.25601199999..."
147,"(14, 3, 1, 15, 0, 1)",3,"[[0.9343629999999999, -0.30702599999999997, 0....","[0.9592529999999999, -0.160622, 0.232454]"
148,"(10, 0, 1, 15, 1, 0)",3,"[[0.9039999999999999, 0.144119, 0.402509], [0....","[0.9592529999999999, 0.160622, 0.232454]"


In [1873]:
dfpenta['type']=0

In [1875]:
dfpenta=dfpenta[dfpenta.columns[[0,3,1,2]]]

In [1876]:
dfpenta

,idx,type,vertices,center
0,"(0, 1, 2, 3, 4)",0,"[(0.123386, 0.08663499999999999, 0.98857), (-0...","[0.0, 0.0, 1.0]"
1,"(0, 1, 11, 16, 10)",0,"[(0.347887, 0.749751, 0.562893), (0.173788, 0....","[0.2777589387431412, 0.8482349260125831, 0.450..."
2,"(1, 2, 12, 17, 11)",0,"[(-0.605552, 0.562546, 0.562893), (-0.721617, ...","[-0.7208870909376458, 0.5262834731781656, 0.45..."
3,"(2, 3, 13, 18, 12)",0,"[(-0.722139, -0.402078, 0.562893), (-0.619772,...","[-0.7232916691387643, -0.5229738982719108, 0.4..."
4,"(3, 4, 14, 19, 13)",0,"[(0.159246, -0.811044, 0.562893), (0.338576999...","[0.2738684794799379, -0.8494989872752924, 0.45..."
5,"(4, 0, 10, 15, 14)",0,"[(0.820558, -0.099175, 0.562893), (0.829024, 0...","[0.892551581527721, -0.0020453458982151408, 0...."
6,"(5, 6, 7, 8, 9)",0,"[(-0.123386, 0.08663499999999999, -0.98857), (...","[0.0, 0.0, -1.0]"
7,"(5, 6, 16, 11, 17)",0,"[(-0.347887, 0.749751, -0.562893), (-0.173788,...","[-0.282522366258839, 0.8495807858689473, -0.44..."
8,"(6, 7, 15, 10, 16)",0,"[(0.605552, 0.562546, -0.562893), (0.721617, 0...","[0.7206950666700204, 0.5312297333272566, -0.44..."
9,"(7, 8, 19, 14, 15)",0,"[(0.722139, -0.402078, -0.562893), (0.619772, ...","[0.7279363832640424, -0.5212628829219892, -0.4..."


In [1877]:
df_row_merged = pd.concat([dfpenta, dfhexa], ignore_index=True)

In [1878]:
df_row_merged 

,idx,type,vertices,center
0,"(0, 1, 2, 3, 4)",0,"[(0.123386, 0.08663499999999999, 0.98857), (-0...","[0.0, 0.0, 1.0]"
1,"(0, 1, 11, 16, 10)",0,"[(0.347887, 0.749751, 0.562893), (0.173788, 0....","[0.2777589387431412, 0.8482349260125831, 0.450..."
2,"(1, 2, 12, 17, 11)",0,"[(-0.605552, 0.562546, 0.562893), (-0.721617, ...","[-0.7208870909376458, 0.5262834731781656, 0.45..."
3,"(2, 3, 13, 18, 12)",0,"[(-0.722139, -0.402078, 0.562893), (-0.619772,...","[-0.7232916691387643, -0.5229738982719108, 0.4..."
4,"(3, 4, 14, 19, 13)",0,"[(0.159246, -0.811044, 0.562893), (0.338576999...","[0.2738684794799379, -0.8494989872752924, 0.45..."
...,...,...,...,...
157,"(15, 2, 1)",1,"[[0.9367709999999999, 0.020336999999999997, -0...","[0.951333, 0.165815, -0.259752]"
158,"(15, 1, 2)",1,"[[0.905531, -0.26689599999999997, -0.329818], ...","[0.95868, -0.12405999999999999, -0.25601199999..."
159,"(14, 3, 1, 15, 0, 1)",3,"[[0.9343629999999999, -0.30702599999999997, 0....","[0.9592529999999999, -0.160622, 0.232454]"
160,"(10, 0, 1, 15, 1, 0)",3,"[[0.9039999999999999, 0.144119, 0.402509], [0....","[0.9592529999999999, 0.160622, 0.232454]"


In [1880]:
arr=[1,2,3,4,5,6]
def swaptmp(x):
    x[3],x[5]=x[5],x[3]
    return x
swaptmp(arr)

[1, 2, 3, 6, 5, 4]

In [1887]:
df_tmp=pd.DataFrame({'A':[[1,2,3,4,5,6],[1,2,3,4,5,6]],'good':[True,False]})

In [1888]:
df_tmp

,A,good
0,"[1, 2, 3, 4, 5, 6]",True
1,"[1, 2, 3, 4, 5, 6]",False


In [1889]:
mask = (df_tmp['good'] == False)
df_tbm = df_tmp[mask]

In [1890]:
df_tbm

,A,good
1,"[1, 2, 3, 4, 5, 6]",False


In [1893]:
df_tmp.loc[mask, 'A']= df_tbm['A'].map(swaptmp)

In [1894]:
df_tmp

,A,good
0,"[1, 2, 3, 4, 5, 6]",True
1,"[1, 2, 3, 6, 5, 4]",False


In [265]:
def expand(x,y):
    return sum((x,y),())


# In[1220]:


fact=0
n=4
xoff7=-1/2
yoff7=0
scale = 1/(n*sqrt(3))


cases = np.zeros((1,4))
#cases[0,0],cases[0,1],cases[0,2],cases[0,3] =  expand(xycent(xoff7,yoff7,0,1,n),(-2*pi/3,3))
#cases[1,0],cases[1,1],cases[1,2],cases[1,3] =  expand(xycent(xoff7,yoff7,1,n-1,n),(2*pi/3,3))
#cases[2,0],cases[2,1],cases[2,2],cases[2,3] =  expand(xycent(xoff7,yoff7,n-1,0,n),(0,3))
#cases[0,0],cases[0,1],cases[0,2],cases[0,3] =  expand(xycent(xoff7,yoff7,0,n-1,n),(-2*pi/3,4))
#cases[0,0],cases[0,1],cases[0,2],cases[0,3] =  expand(xycent(xoff7,yoff7,n-1,1,n),(2*pi/3,4))
print('verif: ',expand(xycent(xoff7,yoff7,1,0,n),(0,4)))
cases[0,0],cases[0,1],cases[0,2],cases[0,3] =  expand(xycent(xoff7,yoff7,1,0,n),(0,4))


fig = plt.figure()
ax = Axes3D(fig)
ax.set_xlabel(r'$X$', fontsize=20)
ax.set_ylabel(r'$Y$', fontsize=20)
ax.set_zlabel(r'$Z$', fontsize=20)    

for k in [0,1,2,3,4]:
#for k in [5,6,7,8,9]:
    a = icoTriangs[k,0]
    b = icoTriangs[k,1]
    c = icoTriangs[k,2]
    print(k,a,b,c)
    pts3d = []
    col = []
    print("cases: ",cases)
    for cas in cases:
        th = cas[2]
        if cas[3] == 3:
            pt2d = np.array([sqrt(3)/2-fact,1/2+fact/sqrt(3)]) # type 3 
            col.append('k')
        else:
            pt2d = np.array([-sqrt(3)/2+fact,1/2+fact/sqrt(3)]) # type 4 
            col.append('r')
            
        rot_mat = scale * np.array([[np.cos(th), -np.sin(th)],
                                        [np.sin(th), np.cos(th)]])
        

        pt2d = np.matmul(rot_mat,pt2d)
        pt2d[0] += cas[0]
        pt2d[1] += cas[1]
        
        print(cas[0],cas[1],pt2d)
        
        pt3d = myround(getProjectedPt(pt2d,icoPoints,a,b,c))

        
        print(pt3d)
        
        pts3d.append(pt3d)

    
    pts3d = np.array(list(pts3d))*tol
    ax.scatter(pts3d[:,0],pts3d[:,1],pts3d[:,2],marker='o',s=15,color=col[:])

#    vertsf=list(zip(pts3d[:,0],pts3d[:,1],pts3d[:,2]))
#    ax.add_collection3d(Poly3DCollection([vertsf], facecolors = 'purple', edgecolors='k', linewidths=1, alpha=0.2))
 
    
ax.set_xlim([-1,1])
ax.set_ylim([-1,1])
ax.set_zlim([-1,1])
plt.show


verif:  (-0.25, 0.0, 0, 4)
0 0 1 2
cases:  [[-0.25  0.    0.    4.  ]]
-0.25 0.0 [-0.375       0.07216878]
(123386, 86635, 988570)
1 0 2 3
cases:  [[-0.25  0.    0.    4.  ]]
-0.25 0.0 [-0.375       0.07216878]
(-44266, 144119, 988570)
2 0 3 4
cases:  [[-0.25  0.    0.    4.  ]]
-0.25 0.0 [-0.375       0.07216878]
(-150744, 2436, 988570)
3 0 4 5
cases:  [[-0.25  0.    0.    4.  ]]
-0.25 0.0 [-0.375       0.07216878]
(-48899, -142614, 988570)
4 0 5 1
cases:  [[-0.25  0.    0.    4.  ]]
-0.25 0.0 [-0.375       0.07216878]
(120523, -90576, 988570)


<function matplotlib.pyplot.show(*args, **kw)>

In [134]:
t0,t1,t2,t3=expand(xycent(-1/2,0,0,1,4),(0,4))

In [135]:
print(t0,t1,t2,t3)

-0.375 0.4330127018922193 0 4


In [133]:
cases

array([[-0.25,  0.  ,  0.  ,  4.  ]])

In [131]:
def xycent(xoff7,yoff7,i,j,n):
    '''
            2D localisation of the center of a pentagon in the frame of a icosaedre face
    '''
    return xoff7+i*1/n+j*1/(2*n), yoff7+j*sqrt(3)/n


In [136]:
n=4
xoff7=-1/2
yoff7=0
scale = 1/(n*sqrt(3))
print('verif: ',expand(xycent(xoff7,yoff7,1,0,n),(0,4)))
cases[0,0],cases[0,1],cases[0,2],cases[0,3] =  expand(xycent(xoff7,yoff7,1,0,n),(0,4))
print(cases)

verif:  (-0.25, 0.0, 0, 4)
[[-0.25  0.    0.    4.  ]]
